In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import re

In [2]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

In [21]:
query = input('검색 키워드를 입력하세요 : ') 
query = query.replace(' ', '+') 

news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : ')) 

검색 키워드를 입력하세요 : 신한은행
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 1000


In [22]:
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')

In [23]:
news_dict = {} 
idx = 0 
cur_page = 1

In [24]:
print()
print('크롤링 중...')

while idx < news_num:
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    
    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    cur_page += 1
    
    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')


크롤링 중...


In [25]:
print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T


크롤링 완료
데이터프레임 변환


In [26]:
news_df

,title,url
0,KB국민은행 2년 연속 경영평가 최우수기업 선정,http://yna.kr/AKR20210522035700003?did=1195m
1,"""방금 앱상담, 만족하셨나요?"" 신한銀, 은행권 첫 '디지털 고객경험 만족도 조사'",https://www.hankyung.com/economy/article/20210...
2,"신한금융, 스타트업 비즈니스 네트워킹 프로그램 개최",http://www.edaily.co.kr/news/newspath.asp?news...
3,“문과생 말고 컴공생 모셔요”… 공채 대신 IT 인력 수시 채용하는 은행들,https://biz.chosun.com/stock/finance/2021/05/2...
4,"'배달통' 가고, 신한은행·티몬 오고… 배달앱 시장 춘추전국시대",http://www.viva100.com/main/view.php?key=20210...
...,...,...
995,"신한은행, 통계청과 빅데이터 기반 사회 문제 공동연구 진행",http://www.nspna.com/news/?mode=view&newsid=50...
996,"신한금융그룹, 정책형 뉴딜펀드 '첫' 투자 실행 완료",http://news.tf.co.kr/read/economy/1856746.htm
997,"신한은행, '신한 S Life Care' 증여신탁 출시",http://www.thebigdata.co.kr/view.php?ud=202105...
998,"신한은행, 학대피해아동쉼터에 차량 전달",http://www.enewstoday.co.kr/news/articleView.h...


In [27]:
news_df.to_csv("gisa.csv")

In [29]:
import pandas as pd

In [53]:
my_title_df = pd.read_csv("gisa.csv")

In [54]:
my_title_df

,Unnamed: 0,title,url
0,0,KB국민은행 2년 연속 경영평가 최우수기업 선정,http://yna.kr/AKR20210522035700003?did=1195m
1,1,"""방금 앱상담, 만족하셨나요?"" 신한銀, 은행권 첫 '디지털 고객경험 만족도 조사'",https://www.hankyung.com/economy/article/20210...
2,2,"신한금융, 스타트업 비즈니스 네트워킹 프로그램 개최",http://www.edaily.co.kr/news/newspath.asp?news...
3,3,“문과생 말고 컴공생 모셔요”… 공채 대신 IT 인력 수시 채용하는 은행들,https://biz.chosun.com/stock/finance/2021/05/2...
4,4,"'배달통' 가고, 신한은행·티몬 오고… 배달앱 시장 춘추전국시대",http://www.viva100.com/main/view.php?key=20210...
...,...,...,...
995,995,"신한은행, 통계청과 빅데이터 기반 사회 문제 공동연구 진행",http://www.nspna.com/news/?mode=view&newsid=50...
996,996,"신한금융그룹, 정책형 뉴딜펀드 '첫' 투자 실행 완료",http://news.tf.co.kr/read/economy/1856746.htm
997,997,"신한은행, '신한 S Life Care' 증여신탁 출시",http://www.thebigdata.co.kr/view.php?ud=202105...
998,998,"신한은행, 학대피해아동쉼터에 차량 전달",http://www.enewstoday.co.kr/news/articleView.h...


In [55]:
my_title_df['label'] = 0

In [56]:
my_title_df

,Unnamed: 0,title,url,label
0,0,KB국민은행 2년 연속 경영평가 최우수기업 선정,http://yna.kr/AKR20210522035700003?did=1195m,0
1,1,"""방금 앱상담, 만족하셨나요?"" 신한銀, 은행권 첫 '디지털 고객경험 만족도 조사'",https://www.hankyung.com/economy/article/20210...,0
2,2,"신한금융, 스타트업 비즈니스 네트워킹 프로그램 개최",http://www.edaily.co.kr/news/newspath.asp?news...,0
3,3,“문과생 말고 컴공생 모셔요”… 공채 대신 IT 인력 수시 채용하는 은행들,https://biz.chosun.com/stock/finance/2021/05/2...,0
4,4,"'배달통' 가고, 신한은행·티몬 오고… 배달앱 시장 춘추전국시대",http://www.viva100.com/main/view.php?key=20210...,0
...,...,...,...,...
995,995,"신한은행, 통계청과 빅데이터 기반 사회 문제 공동연구 진행",http://www.nspna.com/news/?mode=view&newsid=50...,0
996,996,"신한금융그룹, 정책형 뉴딜펀드 '첫' 투자 실행 완료",http://news.tf.co.kr/read/economy/1856746.htm,0
997,997,"신한은행, '신한 S Life Care' 증여신탁 출시",http://www.thebigdata.co.kr/view.php?ud=202105...,0
998,998,"신한은행, 학대피해아동쉼터에 차량 전달",http://www.enewstoday.co.kr/news/articleView.h...,0


In [59]:
my_title_df.to_csv('gisa2.csv')

In [60]:
with open("negative_words_self.txt", encoding='utf-8') as neg:
  negative = neg.readlines() 

with open("positive_words_self.txt", encoding='utf-8') as pos:
  positive = pos.readlines()
  
negative = [neg.replace("\n", "") for neg in negative]
positive = [pos.replace("\n", "") for pos in positive]

In [66]:
from tqdm import tqdm 
import re 
labels = [] 
title_data = list(my_title_df['title']) 
for title in tqdm(title_data): 
    clean_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', '', title) 
    negative_flag = False 
    label = 0 
    for i in range(len(negative)): 
        if negative[i] in clean_title: 
            label = -1 
            negative_flag = True 
            print("negative 비교단어 : ", negative[i], "clean_title : ", clean_title) 
            break 
    if negative_flag == False: 
        for i in range(len(positive)):
            if positive[i] in clean_title: 
                label = 1 
                print("positive 비교단어 : ", positive[i], "clean_title : ", clean_title) 
                break 
    labels.append(label)
print(labels)

100%|██████████| 1000/1000 [00:00<00:00, 11822.13it/s]

positive 비교단어 :  출시 clean_title :  신한은행 자산관리플랫폼 SOL 지갑 출시백신 접종 증명 기능도
positive 비교단어 :  제휴 clean_title :  시중은행·카뱅 암호화폐 거래소 제휴 꺼리는 이유는
positive 비교단어 :  출시 clean_title :  신한은행 쏠 지갑’ 출시 코로나 백신접종 증명도 가능
positive 비교단어 :  출시 clean_title :  신한은행 선물환 거래확인서 비대면 서비스 출시
positive 비교단어 :  출시 clean_title :  신한은행 디지털 월렛 SOL 지갑 출시코로나 백신접종 증명 가능
positive 비교단어 :  출시 clean_title :  은행들 디지털 자산 시장 선점하자”편의성 높인 디지털 지갑 서비스 출시 러시
positive 비교단어 :  진출 clean_title :  하나은행도 페이먼트 시장 진출GLN 결합 하나페이 나오나
positive 비교단어 :  제휴 clean_title :  KB·하나·우리은행 가상자산 거래소와 실명 계좌 제휴 안할듯
positive 비교단어 :  출시 clean_title :  자산관리에서 백신접종 증명까지신한은행 쏠 지갑 출시
positive 비교단어 :  유치 clean_title :  게임사·빅테크와 맞손 시중은행  MZ세대 유치에 총력
positive 비교단어 :  출시 clean_title :  신한은행 쏠SOL 지갑 출시백신접종 증명서비스도 제공
positive 비교단어 :  출시 clean_title :  신한은행 디지털 월렛 SOL 지갑’ 출시
positive 비교단어 :  체결 clean_title :  KB국민은행 샌드박스 게이밍과 네이밍 스폰서십 체결
positive 비교단어 :  체결 clean_title :  KB국민은행샌드박스 게이밍 네이밍 스폰서십 추가 체결
positive 비교단어 :  체결 clean_title :  KB국민은행 샌드박스 게이밍과 네이밍 스폰서십 추가 체결
positive

In [67]:
my_title_df['label'] = labels



In [68]:
my_title_df

,Unnamed: 0,title,url,label
0,0,KB국민은행 2년 연속 경영평가 최우수기업 선정,http://yna.kr/AKR20210522035700003?did=1195m,0
1,1,"""방금 앱상담, 만족하셨나요?"" 신한銀, 은행권 첫 '디지털 고객경험 만족도 조사'",https://www.hankyung.com/economy/article/20210...,0
2,2,"신한금융, 스타트업 비즈니스 네트워킹 프로그램 개최",http://www.edaily.co.kr/news/newspath.asp?news...,0
3,3,“문과생 말고 컴공생 모셔요”… 공채 대신 IT 인력 수시 채용하는 은행들,https://biz.chosun.com/stock/finance/2021/05/2...,0
4,4,"'배달통' 가고, 신한은행·티몬 오고… 배달앱 시장 춘추전국시대",http://www.viva100.com/main/view.php?key=20210...,0
...,...,...,...,...
995,995,"신한은행, 통계청과 빅데이터 기반 사회 문제 공동연구 진행",http://www.nspna.com/news/?mode=view&newsid=50...,-1
996,996,"신한금융그룹, 정책형 뉴딜펀드 '첫' 투자 실행 완료",http://news.tf.co.kr/read/economy/1856746.htm,0
997,997,"신한은행, '신한 S Life Care' 증여신탁 출시",http://www.thebigdata.co.kr/view.php?ud=202105...,1
998,998,"신한은행, 학대피해아동쉼터에 차량 전달",http://www.enewstoday.co.kr/news/articleView.h...,-1


In [69]:
my_title_df.to_csv('gisa3.csv')